In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import LoraConfig, get_peft_model, PeftConfig
from peft import prepare_model_for_kbit_training
from finetune_data_preprocessing import get_data
from torch import cuda
import torch
import os
ROOT = os.getcwd()


/home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/justinchanchan8/miniconda3/envs/ts did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0'), PosixPath('/usr/local/cuda/lib64/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:


In [2]:
device = 'cuda' if cuda.is_available() else 'cpu'
cuda.empty_cache()
print(device)

cuda


In [5]:
import transformers
from datasets import load_dataset, Dataset
import pandas as pd

def gen_prompt(text_input):
    return f"""
    <human>: {text_input["Term"]}
    <assistant>: {text_input["Definition"]}
    """.strip()

def gen_and_tok_prompt(text_input):
    full_input = gen_prompt(text_input)
    tok_full_prompt = tokenizer(full_input, padding = True , truncation =True)
    return tok_full_prompt

#### Load Finance Fine-tuning Dataset

In [96]:
df_finance = get_data('/home/justinchanchan8/text-summarization/backend/data/finance_vocab.csv')
data = Dataset.from_pandas(df_finance[['Term', 'Definition']])

In [12]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

### Load pretrained Falcon 7B model from model/ directory

In [11]:
# model = AutoModelForCausalLM.from_pretrained(
#     "tiiuae/falcon-7b-instruct",
# #     load_in_8bit=True,
#     device_map='auto',
#     trust_remote_code=True,
# )



Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.13s/it]


In [13]:
model_4bit = AutoModelForCausalLM.from_pretrained(
        "tiiuae/falcon-7b-instruct", 
        device_map="auto",
        quantization_config=quantization_config,
        )

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")
# tokenizer.return_token_type_ids = False

# lora_config = LoraConfig(
#     init_lora_weights=False
# )
# model_4bit.add_adapter(lora_config, adapter_name="adapter_1")

ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        

In [27]:
# model.save_pretrained('/home/justinchanchan8/text-summarization/backend/model')

In [53]:
# tokenizer.pad_token = tokenizer.eos_token
# data = data.map(gen_and_tok_prompt)
# tokenizer.return_token_ids=False

In [94]:
# pipe = pipeline(
#         task="text-generation",
#         model=model_4bit,
#         tokenizer=tokenizer,
#         use_cache=True,
#         device_map="auto",
#         max_length=296,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id,
#         pad_token_id=tokenizer.eos_token_id,
# )

In [95]:
# pipe("""Summarize this article: You're in a perfect environment to learn how to create your own excitement. That feeling you're pining for when you see a text notification? Give that to yourself instead of waiting for other people to give it to you.

# Reflect on and develop your interests. Invest in yourself. Lift. When you cultivate a life that is full, engaging, and interesting, women will want to sign up for that purely on a basis of proximity.

# You need to start clean. You can stop checking your phone not by doing anything physical like tying it to your friend and locking them in a safe, but by identifying the situation that causes you to feel a need for validation. Then, when you can recognize these situations, you can head them off before they can begin.
# """)

[{'generated_text': "Summarize this article: You're in a perfect environment to learn how to create your own excitement. That feeling you're pining for when you see a text notification? Give that to yourself instead of waiting for other people to give it to you.\n\nReflect on and develop your interests. Invest in yourself. Lift. When you cultivate a life that is full, engaging, and interesting, women will want to sign up for that purely on a basis of proximity.\n\nYou need to start clean. You can stop checking your phone not by doing anything physical like tying it to your friend and locking them in a safe, but by identifying the situation that causes you to feel a need for validation. Then, when you can recognize these situations, you can head them off before they can begin.\n\nIf you can't do that, you should be with someone who is more emotionally stable."}]

In [141]:
from peft import prepare_model_for_kbit_training
model_4bit.gradient_checkpointing_enable()
model_4bit = prepare_model_for_kbit_training(model_4bit)

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [109]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [124]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model_4bit = get_peft_model(model_4bit, config)
print_trainable_parameters(model_4bit)

trainable params: 2359296 || all params: 3615822720 || trainable%: 0.06524921664301064


In [111]:
# model = model.to(device)
torch.cuda.empty_cache()

In [121]:
training_args = transformers.TrainingArguments(
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=4,
    logging_steps=25,
    output_dir=ROOT, # give the location where you want to store checkpoints 
    save_strategy='epoch',
    optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.05,
)
trainer = transformers.Trainer(
    model=model_4bit,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model_4bit.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.


IndexError: Invalid key: 38 is out of bounds for size 0

In [46]:
root = os.getcwd()
model.save_pretrained(root)

In [2]:
config = PeftConfig.from_pretrained(root)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
#     load_in_8bit=True,
#     device_map='auto',
    trust_remote_code=True,
)

NameError: name 'root' is not defined

In [53]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    trust_remote_code=True,
)
model_inf = PeftModel.from_pretrained(model,root)

ValueError: Unrecognized configuration class <class 'transformers_modules.tiiuae.falcon-7b-instruct.cf4b3c42ce2fdfe24f753f0f0d179202fea59c99.configuration_falcon.FalconConfig'> to build an AutoTokenizer.
Model type should be one of AlbertConfig, AlignConfig, BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BlipConfig, Blip2Config, BloomConfig, BridgeTowerConfig, CamembertConfig, CanineConfig, ChineseCLIPConfig, ClapConfig, CLIPConfig, CLIPSegConfig, CodeGenConfig, ConvBertConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DebertaConfig, DebertaV2Config, DistilBertConfig, DPRConfig, ElectraConfig, ErnieConfig, ErnieMConfig, EsmConfig, FlaubertConfig, FNetConfig, FSMTConfig, FunnelConfig, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, GPTSanJapaneseConfig, GroupViTConfig, HubertConfig, IBertConfig, JukeboxConfig, LayoutLMConfig,

In [ ]:
prompt = f"""
    <human>: How can i use BDB Data Science LAB?
    <assistant>: 
    """.strip()

# encode the prompt 
encoding = tokenizer(prompt, return_tensors= "pt").to(model.device)

# set teh generation configuration params 
gen_config = model_inf.generation_config
gen_config.max_new_tokens = 200
gen_config.temperature = 0.2
gen_config.top_p = 0.7
gen_config.num_return_sequences = 1
gen_config.pad_token_id = tokenizer.eos_token_id
gen_config.eos_token_id = tokenizer.eos_token_id

# do the inference 
with torch.inference_mode():
    outputs = model.generate(input_ids = encoding.input_ids, attention_mask = encoding.attention_mask,generation_config = gen_config )
print(tokenizer.decode(outputs[0], skip_special_tokens = True ))
